This script mines tweets to train the model.

Use my followers to get tweets, then say similar stuff.

In [2]:
#retweets tweets about the housing crisis

# Import Tweepy, sleep, credentials.py
import tweepy
import time
from datetime import datetime, timezone, timedelta

import sys, os

from authenticate import *

api = get_authenticated_api()

should have already got credentials


## Identify users to RT

In [5]:
starting_user_set = api.friends(count=200)

We could prune these people in a variety of ways. It might help to get a subset where some of them follow each other, then get the users that _they_ follow together and add them to the list. But this might be needlessly complex, so to start, let's just see what we can do with this group.

## Identify tweets from them that we actually want to use

We want:

 - Retweets only - MAYBE, maybe not. I am not sure. It might be OK to just generate random pro-housing slogans.
 - Tweets specific to housing. I would rather avoid a direct keyword approach. Something social, like where they're retweeting each other, might work.
 
Other options:

 - Get those users' favorites using API.favorites

In [24]:
e_g_user = starting_user_set[70]

In [28]:
#dir(e_g_user)

In [26]:

[e_g_user.name,e_g_user. e_g_user.screen_name,e_g_user.status.text]

['Matthew Cutler-Welsh',
 'mcutlerwelsh',
 'Building affordable housing in New Zealand. https://t.co/UVLaNq6JmK\nDo we need fewer rules, or just different ones?… https://t.co/soGZbserYK']

In [103]:
user_statuses = api.user_timeline(
    screen_name="curiouskiwicat",#e_g_user.screen_name,
    include_rts=False,
    tweet_mode='extended', #get full text of 280character tweets https://github.com/tweepy/tweepy/issues/974
    count=200
    )

In [104]:
len(user_statuses)

154

In [105]:
user_retweets = [s for s in user_statuses if s.is_quote_status==True]

In [106]:
len(user_retweets)

32

In [95]:
def describe_status(s):
    print(f"TWEET by {s.user.screen_name}:")
    print(s.full_text)
    #print(", ".join(dir(s)))
    print(f"is_quote_status: {s.is_quote_status}")
    print(f"in_reply_to_screen_name: {s.in_reply_to_screen_name}")
    print("___\n")
    

In [96]:
#",".join(dir(user_retweets[0]))

So we need to look up the original tweet using the quoted_status_id...

In [97]:
for s in user_statuses:
    describe_status(s)
    #identify Retweets with "is_quote_status"
    #get an easy 
    
    

TWEET by curiouskiwicat:
@aniobrien @damienmgrant 🤭
is_quote_status: False
in_reply_to_screen_name: aniobrien
___

TWEET by curiouskiwicat:
@damienmgrant @aniobrien Cultural appropriation, Damien.

you're no true Scotsman
is_quote_status: False
in_reply_to_screen_name: damienmgrant
___

TWEET by curiouskiwicat:
Good.

Defending your home, with a weapon if necessary-well, ok. *Aiming* a gun at a crowd as they're walking on by is threatening behavior, and if you do it, you clearly weren't paying attention in gun school. https://t.co/4SQTUXgSDs
is_quote_status: True
in_reply_to_screen_name: None
___

TWEET by curiouskiwicat:
@andrewtychen I suspect, if we had one ready and rolled out, then relative to right now under complete elimination, people would become more willing to start using it in the event (god forbid) we had a second outbreak.
is_quote_status: False
in_reply_to_screen_name: andrewtychen
___

TWEET by curiouskiwicat:
@hollymathnerd "commitment to diversity" in theory means the

In [100]:
for s in user_retweets:
    describe_status(s)
    #get a tweet, the tweet it retweeted, and the screen names of the authors of both tweets
    original_tweet = api.get_status(s.quoted_status_id,tweet_mode='extended')
    retweet_data = {
        'retweet_text':s.full_text,
        'rt_author':s.user.screen_name,
        'original_text':original_tweet.full_text,
        'original_author':original_tweet.user.screen_name
    }
    print("retweet of:")
    describe_status(original_tweet)

TWEET by curiouskiwicat:
Good.

Defending your home, with a weapon if necessary-well, ok. *Aiming* a gun at a crowd as they're walking on by is threatening behavior, and if you do it, you clearly weren't paying attention in gun school. https://t.co/4SQTUXgSDs
is_quote_status: True
in_reply_to_screen_name: None
___

retweet of:
TWEET by Timcast:
St. Louis couple who aimed guns at protesters charged with felony weapons count

https://t.co/qaURzT0Eag
is_quote_status: False
in_reply_to_screen_name: None
___



In [79]:
retweet_data

{'retweet_text': '"I\'m choosing not to pay you. You got no fucking respect, you\'re only here because of me. But if you\'re not there in the morning, I\'m gonna call Immigration and tell them you\'ve absconded. Look that up. Absconded. It means you\'ve run away." https://t.co/JPoS2kdcI3',
 'rt_author': 'curiouskiwicat',
 'original_text': "Ignore all the political shenanigans today. Just read this one article, which says more about New Zealand today than the acres of reportage and commentary we'll get out of Parliament in the next 24 hrs. Team of 5 million? https://t.co/brkXe6FIHw",
 'original_author': 'bernardchickey'}

get a set of user statuses that are retweets...

get this user's last 10 statuses

In [16]:
dir(e_g_user.status)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'contributors',
 'coordinates',
 'created_at',
 'destroy',
 'entities',
 'favorite',
 'favorite_count',
 'favorited',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'parse',
 'parse_list',
 'place',
 'retweet',
 'retweet_count',
 'retweeted',
 'retweets',
 'source',
 'source_url',
 'text',
 'truncated']

#### Retrieval script to retrieve a set of tweets in my dictionary format and save to a csv

In [107]:
users_to_retrieve = starting_user_set[50:70]

In [111]:
tweet_rt_pairs = []
for u in users_to_retrieve[4:7]:
    print(f"retrieving tweets for user {u.screen_name}")
    user_statuses = api.user_timeline(
    screen_name=u.screen_name,
    include_rts=False,
    tweet_mode='extended', #get full text of 280character tweets https://github.com/tweepy/tweepy/issues/974
    count=150
    )
    
    user_retweets = [s for s in user_statuses if s.is_quote_status==True]
    
    for s in user_retweets[0:10]:
        describe_status(s)
        #get a tweet, the tweet it retweeted, and the screen names of the authors of both tweets
        original_tweet = api.get_status(s.quoted_status_id,tweet_mode='extended')
        retweet_data = {
            'retweet_text':s.full_text,
            'rt_author':s.user.screen_name,
            'original_text':original_tweet.full_text,
            'original_author':original_tweet.user.screen_name
        }
        print("retweet of:")
        describe_status(original_tweet)
        tweet_rt_pairs = tweet_rt_pairs + [retweet_data]

retrieving tweets for user Claire_AKL
TWEET by Claire_AKL:
Sad day at #TeamADO https://t.co/5fOsOIfDJM
is_quote_status: True
in_reply_to_screen_name: None
___

retweet of:
TWEET by GreaterAKL:
@AklCouncil have just disbanded their Auckland Design Office, responsible for things like the shared spaces, the current changes to High St &amp; plans for making city more people friendly. A clear message to council family that if you're innovative &amp; effective you'll be out of a job
is_quote_status: False
in_reply_to_screen_name: AklCouncil
___

TWEET by Claire_AKL:
Polka dots in Nelson :) https://t.co/qtVRJvjBMV
is_quote_status: True
in_reply_to_screen_name: None
___

retweet of:
TWEET by alangraynz:
Taking @Claire_AKL tactical polka dots- but in turf- here in Nelson to help activate a new public realm of a block recovered from cars as a ped mall https://t.co/plqv30yruC
is_quote_status: False
in_reply_to_screen_name: None
___

TWEET by Claire_AKL:
Shortland St zebra works underway 🦓 speedbu

retweet of:
TWEET by AklTransport:
@consindo @consindo Sorry to say, it was for the month of June only! ^HS
is_quote_status: False
in_reply_to_screen_name: consindo
___

TWEET by AklBikes:
Now you will have to go swimming to find the shared path... how is this viable new cycle infrastructure @AklTransport @AklCouncil ?? https://t.co/NLInNX7PEP https://t.co/NQL2QoSCJ8
is_quote_status: True
in_reply_to_screen_name: None
___

retweet of:
TWEET by AklBikes:
New shared path in Kumeu... cant even stay unsubmerged after a bit of downpour... 🤨 https://t.co/UmHhcVROYj
is_quote_status: False
in_reply_to_screen_name: None
___



In [113]:
import pandas as pd

pd.DataFrame(tweet_rt_pairs)

ModuleNotFoundError: No module named 'pandas'

## Save